In [1]:
from codice import mymodel
from codice.mymodel import modello_albero
import pyomo.environ as pyo
import numpy as np
import pandas as pd
from scipy.io.arff import loadarff 
from sklearn.model_selection import train_test_split
from codice.mytree import Tree
import importlib
importlib.reload(mymodel)


SEED = 1

### PREPROCESS DATASET ###

# TODO: NORMALIZZARE DATI
raw_data = loadarff('../dataset/Rice_Cammeo_Osmancik.arff')
df_data = pd.DataFrame(raw_data[0]).to_numpy()

# TODO FIXARE ! SONO TUTTI UNI
x_data_raw = df_data[:, :-1]
# Cammeo = 0, Osmancik = 1
y_data_raw = [0 if x.decode("utf-8") == "Cammeo" else 1 for x in df_data[:,-1]]

X_train, X_test, y_train, y_test = train_test_split(x_data_raw, y_data_raw, test_size=0.95, stratify=y_data_raw, 
                                                        random_state=SEED) 

num_features = len(X_train[0])


In [2]:
### BUILD PARAMETERS ###
alpha = 0.1 # complexity
beta = 5 # n min of elements in a leaf
delta = 2 # depth

# compute epsilon value (uj)
u = [None for _ in range(num_features)]
for j in range(num_features):
    for i in range(len(X_train)-1):
        diff = abs(X_train[i][j] - X_train[i+1][j])
        if (u[j] == None or diff < u[j]) and diff != 0:
            u[j] = diff
umax = max(u)
umin = min(u)

## TODO: capire come calcolarlo
lcap = 0.5 

yi_class = [[None for _ in range(len(np.unique(y_train)))] for _ in range(len(X_train))]
for i in range(len(X_train)):
    for col in range(len(np.unique(y_train))):
        yi_class[i][col] = 1 if y_train[i] == col else 0


mvalue = len(X_train)

print(u)
print(umin)
print(umax)



### BUILD TREE ###
tree = Tree(delta)

print(tree.get_parent(4))

[84.0, 0.0050048828125, 0.6421356201171875, 0.0732269287109375, 1.9848346710205078e-05, 42.0, 0.00022661685943603516]
1.9848346710205078e-05
84.0
2


In [3]:
import importlib
import time
importlib.reload(mymodel)

tstart = time.time()
modello = mymodel.modello_albero(alpha, beta, delta, u, umax, umin, num_features, X_train, y_train, tree, lcap, yi_class, mvalue)
opt = pyo.SolverFactory('gurobi')
results = opt.solve(modello, tee=True) # solves and updates model
pyo.assert_optimal_termination(results)
tend = time.time() - tstart

7
[[2, 1], [1], [3], []]
[None, None, None]
Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-20
Read LP format model from file C:\Users\march\AppData\Local\Temp\tmp54nefgqd.pyomo.lp
Reading time = 0.02 seconds
x1: 2517 rows, 815 columns, 18369 nonzeros
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 9 6900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2517 rows, 815 columns and 18369 nonzeros
Model fingerprint: 0x52f2cba5
Variable types: 3 continuous, 812 integer (796 binary)
Coefficient statistics:
  Matrix range     [5e-01, 2e+04]
  Objective range  [1e-01, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 9 rows and 21 columns
Presolve time: 0.01s
Presolved: 2508 rows, 794 columns, 9225 nonzeros
Variable types: 3 continuous, 791 integer (7

In [4]:
### VALIDAZIONE ###
print(modello.lsmall[4].value )
print(modello.lsmall[5].value )
print(modello.lsmall[6].value )
print(modello.lsmall[7].value )

print()
print(modello.d[1].value )
print(modello.d[2].value )
print(modello.d[3].value )







1.0
1.0
1.0
1.0

1.0
1.0
1.0


In [5]:
print(np.sum(modello.z[i, 4].value for i in range(len(y_train))))
print(np.sum(modello.z[i, 5].value for i in range(len(y_train))))
print(np.sum(modello.z[i, 6].value for i in range(len(y_train))))
print(np.sum(modello.z[i, 7].value for i in range(len(y_train))))

print(modello.n2[4].value)
print(modello.n2[5].value)
print(modello.n2[6].value)
print(modello.n2[7].value)

10.0
32.0
83.0
65.0
10.0
32.0
83.0
65.0


C:\Users\march\AppData\Local\Temp\ipykernel_13748\2236415816.py:1: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  print(np.sum(modello.z[i, 4].value for i in range(len(y_train))))
C:\Users\march\AppData\Local\Temp\ipykernel_13748\2236415816.py:2: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  print(np.sum(modello.z[i, 5].value for i in range(len(y_train))))
C:\Users\march\AppData\Local\Temp\ipykernel_13748\2236415816.py:3: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  print(np.sum(modello.z[i, 6].value for i in range(len(y_train))))
C:\Users\march\AppData\Local\Temp\ipykernel_13748\22364158

In [6]:


print(modello.n2[4].value )
print(modello.n2[5].value )
print(modello.n2[6].value )
print(modello.n2[7].value )

print()
print(modello.n1[0,4].value )
print(modello.n1[0,5].value )
print(modello.n1[0,6].value )
print(modello.n1[0,7].value )

print()
print(modello.c[0,4].value )
print(modello.c[0,5].value )
print(modello.c[0,6].value )
print(modello.c[0,7].value )



10.0
32.0
83.0
65.0

0.0
24.0
8.0
49.0

0.0
1.0
0.0
1.0


In [7]:
print()
print(modello.b[1].value )
print(modello.b[2].value )
print(modello.b[3].value )




0.592190146446228
0.8948467969894409
0.8882372379302979
